In [1]:
import matplotlib.pyplot as plt
import numpy as np

from firedrake import *
from netgen.occ import *
from ngsPETSc import NetgenHierarchy
from firedrake.pyplot import tricontourf
from firedrake.pyplot import triplot

ModuleNotFoundError: No module named 'netgen'

In [ ]:
def solve_navierstokes(mesh,sp,p,nu):

  n = FacetNormal(mesh)
  (x, y) = SpatialCoordinate(mesh)

  # Define Taylor--Hood function space W
  V = VectorFunctionSpace(mesh, "CG", 2)
  Q = FunctionSpace(mesh, "CG", 1)
  W = MixedFunctionSpace([V, Q])

  p0=10/13-x/13 #1atleft,0atright
  bcs = DirichletBC(W.sub(0), Constant((0, 0)), walls)

  # Define Function and TestFunction(s)
  w = Function(W, name="Solution")
  (u, p) = split(w)
  (v, q) = split(TestFunction(W))

  # Define variational form
  F = (
        inner(2*nu*sym(grad(u)), sym(grad(v)))*dx
        - div(u)*q*dx
        - div(v)*p*dx
        + p0*dot(v,n)*ds
        - inner(outer(u, u), sym(grad(v)))*dx
        + inner(dot(outer(u, u), n), v)*ds
        )

  # Solve problem
  solve(F == 0, w, bcs)

  # Save solutions
  (u_, p_) = w.subfunctions
  u_.rename("Velocity")
  p_.rename("Pressure")

  return u_, p_

In [ ]:
# Define viscosity and bcs
nu = Constant(0.2)

sp = {
    "ksp_type": "cg", # use conjugate gradients
    "ksp_monitor": None, # print info about iteration
    "ksp_rtol": 1.0e-10, # residual relative tolerance
    "pc_type": "mg", # use geometric multigrid
     }

# Mesh: a hole removed from a rectangle
disk = WorkPlane(Axes((0, 0, 0), n=Z, h=X)).Circle(1).Face()
rect = WorkPlane(Axes((-3, -3, 0), n=Z, h=X)).Rectangle(13, 6).Face()
domain = rect - disk

# Label boundaries
domain.edges.name = "wall"  # all default to wall
domain.edges.Min(X).name = "inlet"
domain.edges.Max(X).name = "outlet"
geo = OCCGeometry(domain, dim=2)

ngmesh = geo.GenerateMesh(maxh=1)
ngsolve_mesh = Mesh(ngmesh)

# Create a mesh hierarchy without the 'order' parameter
mh = MeshHierarchy(ngsolve_mesh, 2)
mesh = mh[-1]

# Get wall regions
walls = [i + 1 for (i, name) in enumerate(ngmesh.GetRegionNames(codim=1)) if name == "wall"]

u, p = solve_navierstokes(mesh,sp,1,nu)

In [ ]:
fig = plt.figure(figsize=(11,4))

ax1 = fig.add_subplot(1, 1, 1)
levels = np.linspace(0, 1, 51)
contours = tricontourf(u, levels=levels, axes=ax1, cmap="inferno")
#triplot(mesh,axes=ax1)
fig.colorbar(contours, ax=ax1)